In [1]:
from openai import OpenAI
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate #where we change the AI "personality"
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4-1106-preview"

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/Categorization_AI_experiments


In [5]:
survey_participant_input = "a19i" #enter column name here

UCNets = pd.read_excel("/Users/chrissoria/Documents/Research/UCNets_Classification/data/Raw_Cond_for_Coding_all_waves.xlsx", engine='openpyxl',sheet_name="JOINT_DATA",usecols=[survey_participant_input])
UCNets = UCNets[survey_participant_input].dropna().unique()  # Drop NaN values and get unique elements

survey_participant_responses = '; '.join(str(item) for item in UCNets) #what we will feed to the model

UCNets = pd.DataFrame(UCNets, columns=[survey_participant_input])
UCNets[survey_participant_input] = UCNets[survey_participant_input].astype(str).str.lower()
UCNets[survey_participant_input] = UCNets[survey_participant_input].str.strip()
UCNets = UCNets[UCNets[survey_participant_input] != ''].reset_index(drop=True) #trimming all empty rows

UCNets = UCNets.iloc[:5]

UCNets.head()

,a19i
0,relocated back to east coast - closer to my sons
1,move in together with my partner
2,"out of living with my friends, and into living..."
3,to take a new job in new york city (both becau...
4,wanted to live in my own place outside my pare...


Here, I'm trying to "force" the model to "think" in steps by first A. trying to process the response into its own words and B. having it interact with that object. That is, instead of all steps being given at once, I'm having it think in steps. 

This time, I will have it think in a "chain," where I will have it output a response and then feed that response back to it in a seperate prompt. 

In [6]:
def extract_categories_cove(survey_question, 
                       survey_input,
                       user_model,
                       creativity,
                       categories):
    
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    
    categories_str = "\n".join(f"{i + 1}. {cat}" for i, cat in enumerate(categories))
    cat_num = len(categories)
    category_dict = {str(i+1): "0" for i in range(cat_num)}
    example_JSON = json.dumps(category_dict, indent=4)
    
    link1 = []
    link2 = []
    extracted_jsons = []
    
    for response in survey_input:
        prompt = f"""Categorize this survey response "{response}" into all of the following reasons for moving and select all that apply: \
        {categories_str} \
        Provide your work in JSON format where the number belonging to each category is the key and a 1 if the category is present and a 0 if it is not present as key values."""
        try:
            api_response = client.chat.completions.create(
                model=user_model,
                messages=[
                    {'role': 'user', 
                     'content': prompt}
                ],
                temperature=creativity
            )
            reply = api_response.choices[0].message.content
            print(reply)
            link1.append(reply)

            prompt2 = f"""Thank you for categorizing this survey response, "{response}". \
            Can you double check if there are any categories you might've missed or marked as being present incorrectly? \
            Here are the categories once again: {categories_str} \
            If there are any changes, please output a corrected JSON with the new categorization. \
            If there are no changes, please output the original JSON."""
            print(prompt2)
            

            api_response2 = client.chat.completions.create(
                model=user_model,
                messages=[
                    {"role": "system", "content": f"""You just categorized an answer to the question, "{survey_question}". You are revisiting your initial categorizations for accuracy. Here's what you initially identified: {reply}"""}, 
                    {'role':'assistant', 'content': reply},
                    {'role': 'user', 'content': prompt2}
                ],
                temperature=.25,
            )

            reply2 = api_response2.choices[0].message.content
            link2.append(reply2)
            
            extracted_json = re.findall(r'```json\n(\{.*?\})\n```', reply2, re.DOTALL)
            extracted_json = extracted_json[0].replace('[', '').replace(']', '').replace('\n', '').replace(" ", '').replace("  ", '')
            print(extracted_json)
            extracted_jsons.append(extracted_json)
                
        except Exception as e:
            print(f"An error occurred: {e}")
            link1.append(f"Error processing input: {survey_input}")
            link2.append(f"Error processing response: {reply}")
            
    normalized_data_list = []
    error_lines = []
    
    for i, json_str in enumerate(extracted_jsons):
        try:
            parsed_obj = json.loads(json_str)
            normalized_data_list.append(pd.json_normalize(parsed_obj))
        except json.JSONDecodeError:
            error_lines.append(i)
            continue

    normalized_data = pd.concat(normalized_data_list, ignore_index=True)
    
    categorized_data = pd.DataFrame()
    categorized_data['survey_response'] = survey_input
    categorized_data['link1'] = link1
    categorized_data['link2'] = link2
    categorized_data['json'] = extracted_jsons
    
    categorized_data = pd.concat([categorized_data, normalized_data], axis=1)
    
    return categorized_data

In [7]:
survey_question = "Why did you move?"
survey_input = UCNets['a19i']

creativity = .0

user_categories = ["to start or continue living with with a partner/spouse",
                   "related to the person's job, school or career, including transfers, retirement, a new job, or wanting to be closer to work",
                   "related to their partner's job, school or career, such as transfers, retirement, a new job, wanting to be closer to work",
                   "financial reasons, such as increases in rent, affordability of the current housing expenses, pay raises",
                   "related to family members for various reasons, such as providing support, facilitating care, a child's schooling needs, or a new baby"]

user_model = 'gpt-4-1106-preview'

cove = extract_categories_cove(survey_question, 
                            survey_input, 
                            user_model,
                            creativity,
                            user_categories)

cove.to_csv('data/a19i_cove_categorization_5_cats.csv')

```json
{
  "1": 0,
  "2": 0,
  "3": 0,
  "4": 0,
  "5": 1
}
```
Thank you for categorizing this survey response, "relocated back to east coast - closer to my sons".             Can you double check if there are any categories you might've missed or marked as being present incorrectly?             Here are the categories once again: 1. to start or continue living with with a partner/spouse
2. related to the person's job, school or career, including transfers, retirement, a new job, or wanting to be closer to work
3. related to their partner's job, school or career, such as transfers, retirement, a new job, wanting to be closer to work
4. financial reasons, such as increases in rent, affordability of the current housing expenses, pay raises
5. related to family members for various reasons, such as providing support, facilitating care, a child's schooling needs, or a new baby             If there are any changes, please output a corrected JSON with the new categorization.             If 

In [8]:
cove

,survey_response,link1,link2,json,1,2,3,4,5
0,relocated back to east coast - closer to my sons,"```json\n{\n ""1"": 0,\n ""2"": 0,\n ""3"": 0,\n ...",The initial categorization appears to be corre...,"{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",0,0,0,0,1
1,move in together with my partner,"```json\n{\n ""1"": 1,\n ""2"": 0,\n ""3"": 0,\n ...",The initial categorization appears to be corre...,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":0}",1,0,0,0,0
2,"out of living with my friends, and into living...","```json\n{\n ""1"": 1,\n ""2"": 0,\n ""3"": 0,\n ...",The initial categorization appears to be corre...,"{""1"":1,""2"":0,""3"":0,""4"":0,""5"":0}",1,0,0,0,0
3,to take a new job in new york city (both becau...,"```json\n{\n ""1"": 0,\n ""2"": 1,\n ""3"": 0,\n ...",The initial categorization appears to be corre...,"{""1"":0,""2"":1,""3"":0,""4"":0,""5"":0}",0,1,0,0,0
4,wanted to live in my own place outside my pare...,"```json\n{\n ""1"": 0,\n ""2"": 0,\n ""3"": 0,\n ...","The survey response ""wanted to live in my own ...","{""1"":0,""2"":0,""3"":0,""4"":0,""5"":1}",0,0,0,0,1
